<a href="https://colab.research.google.com/github/93Brun0/github_actions/blob/main/S%26P500BIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === Step 1: Setup ===
import pandas as pd
import requests
import time

# Mount Google Drive (optional, if you want to save to your Drive)
# from google.colab import drive
# drive.mount('/content/drive')

# === Step 2: Load ticker list (local fallback ready) ===
try:
    sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp500_df = pd.read_html(sp500_url)[0]
    print("✅ Wikipedia fetch successful.")
except:
    from google.colab import files
    uploaded = files.upload()  # Upload sp500_tickers.csv from your computer
    sp500_df = pd.read_csv('sp500_tickers.csv')
    print("⚠️ Wikipedia failed — using uploaded CSV.")

tickers = sp500_df['Symbol'].str.replace('.', '-', regex=False).tolist()

# === Step 3: Set API key ===
API_KEY = 'wfde30s3RwhH2KgloTh5F4xc5Fo3TITG'  # Replace with your valid FMP key

# === Step 4: Collection loop ===
all_data = []
skipped = []

for i, ticker in enumerate(tickers):
    print(f"[{i+1}/{len(tickers)}] {ticker}")

    try:
        income = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{ticker}?limit=5&apikey={API_KEY}').json()
        profile = requests.get(f'https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={API_KEY}').json()
        balance = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?limit=5&apikey={API_KEY}').json()

        if not income or not profile or not balance:
            skipped.append((ticker, 'missing data'))
            continue

        for year in income:
            y = year['calendarYear']
            bmatch = next((b for b in balance if b['calendarYear'] == y), None)
            if not bmatch: continue

            ni = year.get('netIncome')
            ta = bmatch.get('totalAssets')
            te = bmatch.get('totalStockholdersEquity')

            all_data.append({
                'Ticker': ticker,
                'Year': y,
                'Net Income': ni,
                'Total Assets': ta,
                'Total Equity': te,
                'ROA': ni/ta if ta else None,
                'ROE': ni/te if te else None,
                'Company': profile[0].get('companyName'),
                'Revenue': year.get('revenue'),
                'EPS': year.get('eps'),
                'Sector': profile[0].get('sector'),
                'Industry': profile[0].get('industry'),
            })

    except Exception as e:
        skipped.append((ticker, str(e)))
        continue

    if (i+1) % 50 == 0:
        df_checkpoint = pd.DataFrame(all_data)
        df_checkpoint.to_csv('checkpoint.csv', index=False)
        print("✅ Saved checkpoint.")

    time.sleep(1.2)

# === Step 5: Save everything ===
df_final = pd.DataFrame(all_data)
df_final.to_csv('sp500_fundamentals_colab.csv', index=False)
pd.DataFrame(skipped, columns=['Ticker', 'Reason']).to_csv('skipped_tickers_colab.csv', index=False)

print("✅ All done.")


✅ Wikipedia fetch successful.
[1/503] MMM
[2/503] AOS
[3/503] ABT
[4/503] ABBV
[5/503] ACN
[6/503] ADBE
[7/503] AMD
[8/503] AES
[9/503] AFL
[10/503] A
[11/503] APD
[12/503] ABNB
[13/503] AKAM
[14/503] ALB
[15/503] ARE
[16/503] ALGN
[17/503] ALLE
[18/503] LNT
[19/503] ALL
[20/503] GOOGL
[21/503] GOOG
[22/503] MO
[23/503] AMZN
[24/503] AMCR
[25/503] AEE
[26/503] AEP
[27/503] AXP
[28/503] AIG
[29/503] AMT
[30/503] AWK
[31/503] AMP
[32/503] AME
[33/503] AMGN
[34/503] APH
[35/503] ADI
[36/503] ANSS
[37/503] AON
[38/503] APA
[39/503] APO
[40/503] AAPL
[41/503] AMAT
[42/503] APTV
[43/503] ACGL
[44/503] ADM
[45/503] ANET
[46/503] AJG
[47/503] AIZ
[48/503] T
[49/503] ATO
[50/503] ADSK
[51/503] ADP
[52/503] AZO
[53/503] AVB
[54/503] AVY
[55/503] AXON
[56/503] BKR
[57/503] BALL
[58/503] BAC
[59/503] BAX
[60/503] BDX
[61/503] BRK-B
[62/503] BBY
[63/503] TECH
[64/503] BIIB
[65/503] BLK
[66/503] BX
[67/503] BK
[68/503] BA
[69/503] BKNG
[70/503] BSX
[71/503] BMY
[72/503] AVGO
[73/503] BR
[74/503] BRO

In [ ]:
from google.colab import files
files.download('sp500_fundamentals_colab.csv')
files.download('skipped_tickers_colab.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# === Step 1: Setup ===
import pandas as pd
import requests
import time

# === Step 2: Load ticker list ===
try:
    sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp500_df = pd.read_html(sp500_url)[0]
    print("✅ Wikipedia fetch successful.")
except:
    from google.colab import files
    uploaded = files.upload()  # Upload sp500_tickers.csv manually if needed
    sp500_df = pd.read_csv('sp500_tickers.csv')
    print("⚠️ Wikipedia failed — using uploaded CSV.")

tickers = sp500_df['Symbol'].str.replace('.', '-', regex=False).tolist()

# === Step 3: Set your FMP API key ===
API_KEY = '4xEiP6KUhNgHm9kqAsQeAZoSCzvUIBo4'  # Replace with your actual key

# === Step 4: Main data collection loop ===
all_data = []
skipped = []

for i, ticker in enumerate(tickers):
    print(f"[{i+1}/{len(tickers)}] Processing {ticker}")

    try:
        # Get income statement
        income = requests.get(
            f'https://financialmodelingprep.com/api/v3/income-statement/{ticker}?limit=5&apikey={API_KEY}'
        ).json()
        if not isinstance(income, list) or len(income) == 0:
            skipped.append((ticker, 'income - invalid or empty'))
            continue

        # Get profile
        profile = requests.get(
            f'https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={API_KEY}'
        ).json()
        if not isinstance(profile, list) or len(profile) == 0:
            skipped.append((ticker, 'profile - invalid or empty'))
            continue
        profile = profile[0]

        # Get balance sheet
        balance = requests.get(
            f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?limit=5&apikey={API_KEY}'
        ).json()
        if not isinstance(balance, list) or len(balance) == 0:
            skipped.append((ticker, 'balance - invalid or empty'))
            continue

        # Match by year
        for year in income:
            y = year.get('calendarYear')
            bmatch = next((b for b in balance if b.get('calendarYear') == y), None)
            if not bmatch:
                continue

            ni = year.get('netIncome')
            ta = bmatch.get('totalAssets')
            te = bmatch.get('totalStockholdersEquity')

            all_data.append({
                'Ticker': ticker,
                'Year': y,
                'Net Income': ni,
                'Total Assets': ta,
                'Total Equity': te,
                'ROA': ni/ta if ta else None,
                'ROE': ni/te if te else None,
                'Company': profile.get('companyName'),
                'Revenue': year.get('revenue'),
                'EPS': year.get('eps'),
                'Sector': profile.get('sector'),
                'Industry': profile.get('industry'),
            })

    except Exception as e:
        skipped.append((ticker, f'exception: {str(e)}'))
        continue

    if (i + 1) % 50 == 0:
        pd.DataFrame(all_data).to_csv('checkpoint.csv', index=False)
        print("💾 Saved checkpoint")

    time.sleep(1.2)

# === Step 5: Save results ===
pd.DataFrame(all_data).to_csv('sp500_fundamentals_colab.csv', index=False)
pd.DataFrame(skipped, columns=['Ticker', 'Reason']).to_csv('skipped_tickers_colab.csv', index=False)

print("✅ All done.")


✅ Wikipedia fetch successful.
[1/503] Processing MMM
[2/503] Processing AOS
[3/503] Processing ABT
[4/503] Processing ABBV
[5/503] Processing ACN
[6/503] Processing ADBE
[7/503] Processing AMD
[8/503] Processing AES
[9/503] Processing AFL
[10/503] Processing A
[11/503] Processing APD
[12/503] Processing ABNB
[13/503] Processing AKAM
[14/503] Processing ALB
[15/503] Processing ARE
[16/503] Processing ALGN
[17/503] Processing ALLE
[18/503] Processing LNT
[19/503] Processing ALL
[20/503] Processing GOOGL
[21/503] Processing GOOG
[22/503] Processing MO
[23/503] Processing AMZN
[24/503] Processing AMCR
[25/503] Processing AEE
[26/503] Processing AEP
[27/503] Processing AXP
[28/503] Processing AIG
[29/503] Processing AMT
[30/503] Processing AWK
[31/503] Processing AMP
[32/503] Processing AME
[33/503] Processing AMGN
[34/503] Processing APH
[35/503] Processing ADI
[36/503] Processing ANSS
[37/503] Processing AON
[38/503] Processing APA
[39/503] Processing APO
[40/503] Processing AAPL
[41/503]

In [ ]:
from google.colab import files
files.download('sp500_fundamentals_colab.csv')
files.download('skipped_tickers_colab.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>